## Context Managers

### Documentation

https://docs.python.org/3.7/library/contextlib.html?highlight=context%20manager#

http://book.pythontips.com/en/latest/context_managers.html


### Recap

In [1]:
with open('hello.txt', 'w') as f:
    f.write('hello, world!')

In [2]:
# same as the above

f = open('hello.txt', 'w')
try:
    f.write('hello, world')
finally:
    f.close()

In [3]:
# the below implementation won’t guarantee the file is closed if there’s an exception during the f.write() call

f = open('hello.txt', 'w')
f.write('hello, world')
f.close()

### Implementing a Context Manager as a Class

In [4]:
# at the very least a context manager has an __enter__ and __exit__ method defined
# simple implementation of the open() context manager

class ManagedFile:
    def __init__(self, name):
        self.name = name

    def __enter__(self):
        self.file = open(self.name, 'w')
        return self.file

    def __exit__(self, exc_type, exc_val, exc_tb):
        if self.file:
            self.file.close()

In [5]:
# just by defining __enter__ and __exit__ methods we can use our new class in a with statement. 

with ManagedFile('hello1') as f:
    f.write('hello, world!\n')
    f.write('bye now')

### Handling Exceptions

In [6]:
# rewrite __exit__()

class ManagedFile:
    def __init__(self, name):
        self.name = name

    def __enter__(self):
        self.file = open(self.name, 'w')
        return self.file

    # __exit__ method handles the exception
    def __exit__(self, exc_type, exc_val, exc_tb):
        if self.file:
            print("Exception has been handled")
#             print('exc_type: ', exc_type) 
#             print('exc_val: ', exc_val)
#             print('exc_tb: ', exc_tb)
            self.file.close()
            return True

In [7]:
with ManagedFile('hello2') as f:
    f.undefined_function() # non supported method

Exception has been handled


### Implememting a Context Manager as a function decorator

In [8]:
# use as a decorator to define a generator-based factory function for a resource 
# that will then automatically support the with statement.

from contextlib import contextmanager


@contextmanager
def managed_file(name):
    try:
        f = open(name, 'w')
#         print(type(f))
#         print(f)
        yield f
    finally:
        f.close()

In [9]:
with managed_file('hello3') as f:
    f.write('hello, world!\n')
    f.write('bye now')

### Writing pretty APIs with Context Managers

In [10]:
class Indenter:
    def __init__(self):
        self.level = 0

    def __enter__(self):
        self.level += 1
        return self
    
    def __exit__(self, exc_type, exc_val, exc_tb): 
        self.level -= 1

    def print(self, text):
        print('    ' * self.level + text)

In [11]:
with Indenter() as indent:
    indent.print('hi!')
    with indent:
        indent.print('hello')
        with indent:
            indent.print('bark bark')
    indent.print('hey')

    hi!
        hello
            bark bark
    hey


In [12]:
# Inherited or Sub class (Note Indenter in parenthesis)

class HtmlIndenter(Indenter):
    def __init__(self):
        Indenter.__init__(self)

    def print(self, text):
        print('  ' * self.level + text)

In [13]:
with HtmlIndenter() as html_indent:
    html_indent.print('<body>')
    with html_indent:
        html_indent.print('<h3>')
        html_indent.print('la la la')
        html_indent.print('</h3>')
        with html_indent:
            html_indent.print('<div>')
            html_indent.print('blah blah blah')
            html_indent.print('</div>')
    html_indent.print('</body>')

  <body>
    <h3>
    la la la
    </h3>
      <div>
      blah blah blah
      </div>
  </body>
